## AT&T 5G Coverage
This notebook extracts geopackages from the FCC's Broadband Data Collection portal consisting of level 9 h3 hexagons using their public API. Rather than storing that enormous amount of data we extract the attribute table and later use the h3 library to recreate only what we need. 

While functional these notebooks were built in pieces and need to be cleaned up. 

In [ ]:
import requests
import json
import os
from dotenv import load_dotenv

def list_availability_data(date, username, hash_value, verify_ssl=True):
    """
    List FCC availability data for a specific date.
    
    Parameters:
    -----------
    date : str
        Date in format 'YYYY-MM-DD' (e.g., '2024-06-30')
    username : str
        FCC API username
    hash_value : str
        FCC API hash value
    verify_ssl : bool
        Whether to verify SSL certificates (default True)
    
    Returns:
    --------
    dict : API response data
    """
    
    url = f'https://bdc.fcc.gov/api/public/map/downloads/listAvailabilityData/{date}'
    
    headers = {
        'username': username,
        'hash_value': hash_value,
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Accept': 'application/json',
        'Connection': 'close'  # Don't keep connection alive
    }
    
    # Check that we're passing the correct variables
    print(f"Requesting availability data for {date}...")
    print(f"URL: {url}")
    print(f"Username: {username[:3]}***")
    
    # Create session with retry logic
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(
        max_retries=requests.adapters.Retry(
            total=3,
            backoff_factor=1,
            status_forcelist=[500, 502, 503, 504]
        )
    )
    session.mount('https://', adapter)
    session.mount('http://', adapter)
    
    try:
        response = session.get(
            url, 
            headers=headers, 
            timeout=(10, 30),  # (connect, read) timeout
            verify=verify_ssl
        )
        
        # Should probably only print this verification once
        print(f"Status Code: {response.status_code}")
        print(f"Response Headers: {dict(response.headers)}")
        
        response.raise_for_status()
        
        data = response.json()
        
        print(f"\nSuccess! Retrieved data:")
        print(json.dumps(data, indent=2))
        
        return data
        
    except requests.exceptions.HTTPError as e:
        print(f"\nHTTP Error: {e}")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.text}")
        raise
    except requests.exceptions.ConnectionError as e:
        print(f"\nConnection Error: {e}")
        print("\nTroubleshooting tips:")
        print("1. Verify your username and hash_value are correct")
        print("2. Check if you need to register/activate API access with FCC")
        print("3. Try accessing the URL in a browser first")
        print("4. Check if there's a firewall blocking the connection")
        print("5. Try with verify_ssl=False if SSL is the issue")
        raise
    except requests.exceptions.RequestException as e:
        print(f"\nRequest Error: {e}")
        raise
    except json.JSONDecodeError as e:
        print(f"\nJSON Decode Error: {e}")
        print(f"Response text: {response.text}")
        raise
    finally:
        session.close()


if __name__ == "__main__":
    # Configuration
    DATE = "2025-06-30"
    USERNAME   = os.environ["FCC_USERNAME"]
    HASH_VALUE = os.environ["FCC_HASH_VALUE"]
    # Get the list
    result = list_availability_data(
        date=DATE,
        username=USERNAME,
        hash_value=HASH_VALUE
    )


In [ ]:
# Filter records with category='Provider' and subcategory='Hexagon Coverage'
# Combine this with the above cell to prevent "result" from clearing

filtered_records = [
    record for record in result['data']
    if record.get('category') == 'Provider' 
    and record.get('subcategory') == 'Hexagon Coverage'
]

print(f"Found {len(filtered_records)} records")
filtered_records

Found 1189 records


[{'file_id': 1442447,
  'category': 'Provider',
  'subcategory': 'Hexagon Coverage',
  'technology_type': 'Mobile Broadband',
  'technology_code': '300',
  'technology_code_desc': '3G',
  'speed_tier': None,
  'state_fips': '60',
  'state_name': 'American Samoa',
  'provider_id': '170012',
  'provider_name': 'American Samoa Telecom, LLC',
  'file_type': 'gis',
  'file_name': 'bdc_60_170012_3G_mobile_broadband_h3_J25_03feb2026',
  'record_count': '1072'},
 {'file_id': 1442444,
  'category': 'Provider',
  'subcategory': 'Hexagon Coverage',
  'technology_type': 'Mobile Broadband',
  'technology_code': '300',
  'technology_code_desc': '3G',
  'speed_tier': None,
  'state_fips': '72',
  'state_name': 'Puerto Rico',
  'provider_id': '170001',
  'provider_name': 'Puerto Rico Telephone Company, Inc.',
  'file_type': 'gis',
  'file_name': 'bdc_72_170001_3G_mobile_broadband_h3_J25_03feb2026',
  'record_count': '71486'},
 {'file_id': 1442294,
  'category': 'Provider',
  'subcategory': 'Hexagon Co

In [ ]:
import pandas as pd
import os

# Convert to DataFrame and save as CSV
df = pd.DataFrame(filtered_records)
filename = "mobile_datasets.csv"
pathout = "scraped_data\\mno" # Update this storage location
df_filtered = df[
    (df["state_name"].isin(["Pennsylvania", "New York", "New Jersey"])) & # extract our states of interest
    (df["technology_code_desc"] == "5G-NR") & # limit the tech
    (df["provider_name"] == 'AT&T Inc.') # single out a provider 
]

df_filtered.to_csv(os.path.join(pathout, filename), index=False)

print(f"Saved {len(df_filtered)} records to provider_hexagon_coverage.csv")
df_filtered.head()



Saved 6 records to provider_hexagon_coverage.csv


,file_id,category,subcategory,technology_type,technology_code,technology_code_desc,speed_tier,state_fips,state_name,provider_id,provider_name,file_type,file_name,record_count
525,1442799,Provider,Hexagon Coverage,Mobile Broadband,500,5G-NR,35/3,36,New York,130077,AT&T Inc.,gis,bdc_36_130077_5GNR_35_3_mobile_broadband_h3_J2...,458950
536,1442770,Provider,Hexagon Coverage,Mobile Broadband,500,5G-NR,35/3,34,New Jersey,130077,AT&T Inc.,gis,bdc_34_130077_5GNR_35_3_mobile_broadband_h3_J2...,106481
547,1442757,Provider,Hexagon Coverage,Mobile Broadband,500,5G-NR,35/3,42,Pennsylvania,130077,AT&T Inc.,gis,bdc_42_130077_5GNR_35_3_mobile_broadband_h3_J2...,537862
806,1442513,Provider,Hexagon Coverage,Mobile Broadband,500,5G-NR,7/1,42,Pennsylvania,130077,AT&T Inc.,gis,bdc_42_130077_5GNR_7_1_mobile_broadband_h3_J25...,667395
811,1442504,Provider,Hexagon Coverage,Mobile Broadband,500,5G-NR,7/1,34,New Jersey,130077,AT&T Inc.,gis,bdc_34_130077_5GNR_7_1_mobile_broadband_h3_J25...,155273


In [ ]:
# Stepping stone create the list of ID's necessary for the url

import csv

pathout = "scraped_data\\mno" # Update this storage location to match above

with open(os.path.join(pathout, filename),'r') as f:
    m = list(csv.reader(f))

list_of_urls = list()

for i,x in enumerate(m):
    if i > 0:
        list_of_urls.append(x[0])

In [ ]:
import requests
import zipfile
import os
from pathlib import Path
import geopandas as gpd
import tempfile
import pandas as pd
from datetime import datetime
import traceback

OUT_DIR = Path("../scraped_data/mno/att")

def download_and_process_fcc_data(file_id, username, hash_value, out_dir=OUT_DIR):
    download_dir = Path(tempfile.mkdtemp())
    zip_path     = download_dir / f"data_{file_id.replace('/', '_')}.zip"
    extract_path = download_dir / f"extracted_{file_id.replace('/', '_')}"
    out_dir      = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    try:
        # Step 1: Download
        print(f"Downloading file {file_id}...")
        url = f'https://bdc.fcc.gov/api/public/map/downloads/downloadFile/availability/{file_id}/2'
        headers = {
            'username': username,
            'hash_value': hash_value,
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'application/json',
            'Connection': 'close'
        }

        session = requests.Session()
        session.mount('https://', requests.adapters.HTTPAdapter(max_retries=3))
        response = session.get(url, headers=headers, stream=True, timeout=(30, 300))
        response.raise_for_status()

        total_size = int(response.headers.get('content-length', 0))
        downloaded = 0
        with open(zip_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024*1024):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    if total_size:
                        print(f"Progress: {(downloaded/total_size)*100:.1f}% ({downloaded/(1024*1024):.1f} MB)", end='\r')
        print(f"\nDownloaded ({downloaded/(1024*1024):.1f} MB)")

        # Step 2: Extract
        print("Extracting...")
        os.makedirs(extract_path, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        # Step 3: Find geopackage
        gpkg_files = list(extract_path.glob("*.gpkg"))
        if not gpkg_files:
            raise FileNotFoundError("No .gpkg file found in extracted data")
        print(f"Found geopackage: {gpkg_files[0].name}")

        # Step 4: Read and drop geometry
        gdf = gpd.read_file(gpkg_files[0])
        df  = gdf.drop(columns="geometry")
        print(f"Loaded {len(df)} rows, {len(df.columns)} columns")

        # Step 5: Save to CSV
        csv_path = out_dir / f"{file_id.replace('/', '_')}.csv"
        df.to_csv(csv_path, index=False)
        print(f"Saved to {csv_path}")

        return {
            'file_id': file_id,
            'status': 'success',
            'rows': len(df),
            'columns': len(df.columns),
            'file_size_mb': downloaded / (1024*1024),
            'csv_path': str(csv_path),
            'error_message': None,
            'timestamp': datetime.now().isoformat()
        }

    except Exception as e:
        print(f"ERROR processing {file_id}: {e}")
        return {
            'file_id': file_id,
            'status': 'failed',
            'rows': 0,
            'columns': 0,
            'file_size_mb': 0,
            'csv_path': None,
            'error_message': str(e),
            'error_trace': traceback.format_exc(),
            'timestamp': datetime.now().isoformat()
        }

    finally:
        import shutil
        if zip_path.exists():    zip_path.unlink()
        if extract_path.exists(): shutil.rmtree(extract_path)

# Serves two purposes, saves progress in case of mishap during the long process and 
# shows imitation progress bar
def process_batch_downloads(list_of_urls, username, hash_value, out_dir=OUT_DIR):
    results = []
    total   = len(list_of_urls)
    results_csv = Path(out_dir) / "download_results.csv"

    print(f"Starting batch download of {total} files → {out_dir}")
    print("=" * 80)

    for idx, file_id in enumerate(list_of_urls, 1):
        print(f"\n[{idx}/{total}] Processing: {file_id}")
        print("-" * 80)

        result = download_and_process_fcc_data(file_id, username, hash_value, out_dir)
        results.append(result)

        # Save results log after each file
        pd.DataFrame(results).to_csv(results_csv, index=False)
        print(f"Status: {result['status'].upper()}")

    df_results = pd.DataFrame(results)
    print("\n" + "=" * 80)
    print(f"Complete — {len(df_results[df_results['status']=='success'])}/{total} succeeded")
    print(f"Total rows saved: {df_results['rows'].sum()}")
    print(f"Results log: {results_csv}")

    failed = df_results[df_results['status'] == 'failed']
    if len(failed):
        print("\nFailed files:")
        for _, row in failed.iterrows():
            print(f"  - {row['file_id']}: {row['error_message']}")

    return df_results


# Usage
USERNAME   = os.environ["FCC_USERNAME"]
HASH_VALUE = os.environ["FCC_HASH_VALUE"]

results_df = process_batch_downloads(
    list_of_urls=list_of_urls,
    username=USERNAME,
    hash_value=HASH_VALUE
)


Starting batch download of 6 files → ..\scraped_data\mno\att

[1/6] Processing: 1442799
--------------------------------------------------------------------------------
Progress: 100.0% (38.1 MB)
Downloaded (38.1 MB)
Extracting...
Found geopackage: bdc_36_130077_5GNR_35_3_mobile_broadband_h3_J25_03feb2026.gpkg
Loaded 458950 rows, 9 columns
Saved to ..\scraped_data\mno\att\1442799.csv
Status: SUCCESS

[2/6] Processing: 1442770
--------------------------------------------------------------------------------
Progress: 100.0% (8.7 MB)
Downloaded (8.7 MB)
Extracting...
Found geopackage: bdc_34_130077_5GNR_35_3_mobile_broadband_h3_J25_03feb2026.gpkg
Loaded 106481 rows, 9 columns
Saved to ..\scraped_data\mno\att\1442770.csv
Status: SUCCESS

[3/6] Processing: 1442757
--------------------------------------------------------------------------------
Progress: 100.0% (44.2 MB)
Downloaded (44.2 MB)
Extracting...
Found geopackage: bdc_42_130077_5GNR_35_3_mobile_broadband_h3_J25_03feb2026.gpkg
Loaded

In [ ]:
import h3
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from pathlib import Path

att_dir  = Path("../scraped_data/mno/att")
h3_dir   = Path("../scraped_data/mno/att/h3")
h3_dir.mkdir(parents=True, exist_ok=True)

# Load and combine all 6 data CSVs (skip the results log)
csvs = [f for f in att_dir.glob("*.csv") if f.name != "download_results.csv"]
df = pd.concat([pd.read_csv(f) for f in csvs], ignore_index=True)
print(f"Loaded {len(df):,} rows from {len(csvs)} files")
print(f"Technologies: {df['technology'].unique()}")

# Aggregate res9 — one row per hex, summarise across providers/technologies
def aggregate_h3(df, h3_col, target_res=None):
    if target_res is not None:
        df = df.copy()
        df[h3_col] = df[h3_col].apply(lambda h: h3.cell_to_parent(h, target_res))
    # attaches attributes
    return df.groupby(h3_col).agg(
        provider_count = (df.columns[df.columns.str.contains('provider|frn')][0], 'nunique'),
        tech_count     = ('technology', 'nunique'),
        avg_mindown    = ('mindown', 'mean'),
        avg_minup      = ('minup', 'mean'),
        avg_minsignal  = ('minsignal', 'mean'),
        hex_count      = (h3_col, 'count')
    ).reset_index()

# Build and export each resolution
resolutions = [
    # ("res9", None,  9),   # base data, no parent conversion
    ("res8", 8,     8),
    ("res7", 7,     7),
    ("res6", 6,     6),
    # ("res5", 5,     5),
]

for label, target_res, res_num in resolutions:
    print(f"\nAggregating {label}...")
    
    h3_col = "h3_res9_id"
    agg    = aggregate_h3(df, h3_col, target_res)
    agg    = agg.rename(columns={h3_col: "h3id"})
    
    # Convert H3 IDs to polygons
    gdf = gpd.GeoDataFrame(
        agg,
        geometry=[Polygon([(lng, lat) for lat, lng in h3.cell_to_boundary(hid)]) for hid in agg["h3id"]],
        crs="EPSG:4326"
    )
    
    out_path = h3_dir / f"att_{label}.geojson"
    gdf.to_file(out_path, driver="GeoJSON")
    print(f"  Saved {len(gdf):,} hexes → {out_path}")


Loaded 2,587,254 rows from 6 files
Technologies: [500]

Aggregating res8...
  Saved 277,857 hexes → ..\scraped_data\mno\att\h3\att_res8.geojson

Aggregating res7...
  Saved 46,075 hexes → ..\scraped_data\mno\att\h3\att_res7.geojson

Aggregating res6...
  Saved 7,176 hexes → ..\scraped_data\mno\att\h3\att_res6.geojson


then run in the other notebook